In [ ]:
import inspect
import os
import sys
import bookutils
import ipywidgets as widgets

In [ ]:
from Tracer import Tracer
from bookutils import input,next_inputs
from pythonfile import py_fun1,py_fun2
import re

In [ ]:
class State:
    def __init__(self,event,abs_lineno,code,changed_vars,call_stack,frame,rel_lineno,fun_name,file,fun_code):
        self.event = event
        self.abs_lineno = abs_lineno
        self.rel_lineno = rel_lineno
        self.code = code
        self.changed_vars = changed_vars
        self.call_stack = call_stack
        self.frame = frame
        self.stackdepth = None
        self.fun_name = fun_name
        self.file = file
        self.fun_code = fun_code

    def __str__(self):
        return 'Line: ' + str(self.abs_lineno) + ' ('+ str(self.rel_lineno)+'), ('+repr(self.event)+") " + \
               repr(self.code) + '   '+ repr(self.changed_vars) + '   ' + repr(self.fun_name) + '  ('+ repr(self.file)+')'

In [ ]:
class Breakpoint:
    def __init__(self,type,line=None,function=None,condition=None,file=None,is_active=True):
        self.type=type
        self.line_number=line
        self.func_name=function
        self.cond_expression = condition
        self.file = file
        self.is_active=is_active
        self.alias = ""

    def break_here(self,step,log,variables,backwards=False):
        if not self.is_active:
            return False

        current_state = log[step]
        if self.type == 'line':
            return current_state.abs_lineno==self.line_number and current_state.file==self.file
        elif self.type == 'func':
            if backwards:
                if step < len(log)-1:
                    prev = log[step+1]
                    return prev.event=="return" and prev.fun_name==self.func_name
                else:
                    return False
            else:
                if step > 1:
                    prev = log[step-1]
                    return prev.event=="call" and prev.fun_name==self.func_name
                else:
                    return False
        elif self.type =='cond':
            if current_state.abs_lineno==self.line_number:
                try:
                    return eval(self.cond_expression,globals(),variables)
                except Exception as err:
                    #self.log(f"{err.__class__.__name__}: {err}")
                    return False
            else:
                return False
        else:
            return False

    def is_active(self):
        return self.is_active

    def enable(self):
        self.is_active=True

    def disable(self):
        self.is_active=False

    def set_alias(self,alias):
        self.alias = alias

    def __str__(self):
        if self.type == 'line':
            return f"line  {repr(self.file)}:{self.line_number}  {self.is_active}"
        elif self.type == 'func':
            return f"func  {repr(self.file)}:{self.func_name}  {self.is_active}"
        elif self.type =='cond':
            return f"cond  {repr(self.file)}:{self.line_number}  {self.is_active}  {repr(self.cond_expression)}"
        else:
            return ""

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
class Tracer(object):
    def __init__(self, file=sys.stdout):
        """Trace a block of code, sending logs to file (default: stdout)"""
        self.original_trace_function = None
        self.file = file
        self.loglist =[]
        self.last_vars = {}
        pass

    def changed_vars(self, new_vars):
        changed = {}
        for var_name in new_vars:
            if (var_name not in self.last_vars or
                    self.last_vars[var_name] != new_vars[var_name]):
                changed[var_name] = new_vars[var_name]
        self.last_vars = new_vars.copy()
        return changed

    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        # print(*objects, sep=sep, end=end, file=self.file, flush=True)
        # self.logger = self.logger + "\n" + " ".join(objects)


        current_state = State(objects[0], objects[1],objects[2],objects[3],None,objects[5],objects[6],objects[7],objects[8],objects[9])
        self.loglist.append(current_state)


    def traceit(self, frame, event, arg):
        """Tracing function. To be overridden in subclasses."""
        self.log_state(frame, event, arg)
        #self.log(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)

    def log_state(self, frame, event, arg):

        changes = self.changed_vars(frame.f_locals)
        module = inspect.getmodule(frame.f_code)
        func_code = None #TODO change
        if module is None:
            source,function_start = inspect.getsourcelines(frame.f_code)
            file = inspect.getsourcefile(frame)
            relative_lineno = frame.f_lineno-function_start + 1
            current_line = source[relative_lineno - 1]
            func_code = source

        else:
            source,function_start = inspect.getsourcelines(module)
            path = inspect.getsourcefile(module)
            _, tail = os.path.split(path)
            file = tail.replace(".py","")
            relative_lineno = frame.f_lineno-function_start + 1
            current_line = source[frame.f_lineno-1]

            for key, data in inspect.getmembers(module,inspect.isfunction):
                code, line=inspect.getsourcelines(data)
                if data.__name__==frame.f_code.co_name:
                    relative_lineno= frame.f_lineno - line +1
                    func_code = code

        if event == 'call':
            self.log(event, frame.f_lineno, frame.f_code.co_name, changes, None, frame, relative_lineno,frame.f_code.co_name,file,func_code)

        if event == 'line':

            self.log(event, frame.f_lineno, current_line, changes, None, frame, relative_lineno,frame.f_code.co_name,file,func_code)

        if event == 'return':
            self.log(event, frame.f_lineno, repr(arg), changes,None, frame, relative_lineno,frame.f_code.co_name,file,func_code)
            self.last_vars = {}  # Delete 'last' variables


    def _traceit(self, frame, event, arg):
        """Internal tracing function."""
        if frame.f_code.co_name == '__exit__':
            # Do not trace our own _exit_() method
            pass
        else:
            self.traceit(frame, event, arg)
        return self._traceit

    def __enter__(self):
        """Called at begin of `with` block. Turn tracing on."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self._traceit)
        return self

    def __exit__(self, tp, value, traceback):
        """Called at begin of `with` block. Turn tracing off."""
        sys.settrace(self.original_trace_function)
        #self.print_log()

    def print_log(self):
        index = 0
        for log in self.loglist:
            #print('Log Entry: ' + str(index))
            #index = index + 1
            print(log)

    def get_log(self):
        return self.loglist

In [ ]:
class Debugger(object):
    def __init__(self):

        self.current_state = None
        self.callstack = None
        self.callstack_depth = None
        self.current_step = 0
        self.stepping = True
        self.breakpoints = dict()
        self.watchpoints = dict()
        self.last_watchpoint = 1 #delete
        self.interact = True
        self.variables = {}

    def __enter__(self):
        with Tracer() as t:
            #py_fun1(5)
            foo()
        self.execution_log = t.get_log()
        t.print_log()
        print("\n")
        print("\n")
        print("\n")
        if len(self.execution_log)<1:
            print("Log is empty")
            return
        self.construct_callstack()
        self.current_state=self.execution_log[self.current_step]
        #self.print_debugger_status()
        self.step_command()
        self.interaction_loop()

    def __exit__(self, exc_type, exc_val, exc_tb):
        pass

    def step(self):
        if self.current_step<len(self.execution_log)-1:
            self.current_step += 1
            self.current_state = self.execution_log[self.current_step]
            self.construct_vars()
            return True
        else:
            return False

    def step_to_line(self):
        self.callstack=None
        self.callstack_depth=None
        init_step = self.current_step
        while self.current_step<len(self.execution_log)-1:
            self.current_step += 1
            self.current_state = self.execution_log[self.current_step]
            if self.current_state.event=="line":
                self.variables=self.construct_vars_at(self.current_step)
                return True
        self.current_step=init_step
        self.current_state=self.execution_log[self.current_step]
        return False

    def backstep_to_line(self):
        init_step = self.current_step
        while self.current_step>1:
            self.current_step -= 1
            self.current_state = self.execution_log[self.current_step]
            if self.current_state.event=="line":
                self.variables=self.construct_vars_at(self.current_step)
                return True
        self.current_step=init_step
        self.current_state=self.execution_log[self.current_step]
        return False


    def step_back(self):
        if self.current_step> 0:
            self.current_step -= 1
            self.current_state = self.execution_log[self.current_step]
            return True
        else:
            return False


    def construct_callstack(self):
        current_parent = None
        callpoints = list()
        current_depth = 0
        for i in range(0,len(self.execution_log)):
            currentexecution = self.execution_log[i]
            currentexecution.parent = current_parent

            if currentexecution.event == 'call':
                current_parent = self.execution_log[i]
                # das hier ist doch einfach 'if current_parent is not None'
                if self.execution_log[i].parent is not None :
                    current_caller = self.execution_log[i-1]
                    callpoints.append(current_caller)
                    current_parent = current_caller
                else:
                    callpoints.append(current_parent)

                current_depth = current_depth + 1

            elif currentexecution.event == 'return':

                current_depth = current_depth - 1
                caller = callpoints[current_depth]
                current_parent = caller.parent

        for execution in self.execution_log:
            if execution.parent:
                print('File '+ str(execution.parent.file)+' ,Line '+str(execution.parent.rel_lineno) +', Absolute Codeline: '+str(execution.abs_lineno))

    def construct_current_callstack(self):
        current_callstack = list()
        current_caller = self.current_state

        while current_caller is not None :

#            print(str(current_caller.parent))
            if current_caller.parent!=current_caller:
                current_callstack.append(current_caller)
                current_caller = current_caller.parent

        current_callstack.reverse()
        return current_callstack


    def where_command(self,arg=""):
        """Print the current call stack"""
        current_callstack = self.construct_current_callstack()
        #        self.log("\n".join([str(s.parent) for s in current_callstack]))
        if self.callstack_depth is None:
            self.callstack_depth = len(current_callstack)

        for i in range(0,len(current_callstack)):
            call = current_callstack[i]
            if i == self.callstack_depth-1:
                self.log('>> File "'+ str(self.current_state.file)+'", Line '+ str(self.current_state.abs_lineno)  + ', in '+str(self.current_state.fun_name))
            else:
                if call:
                    if call.parent:
                        self.log('File "'+str(call.file)+'", Line '+ str(call.abs_lineno)  + ', in '+ str(call.fun_name))
                    else:
                        self.log('File "'+str(call.file)+'", Line '+ str(call.abs_lineno)  + ', in '+ "<module>")


    def get_execution_index(self, execution):
        for i in range(0,len(self.execution_log)):
            if execution==self.execution_log[i]:
                return i


    def up_command(self,arg=""):
        """Move up the call stack towards the callers"""
        self.callstack = self.construct_current_callstack()

        if self.callstack_depth is not None:
            if self.callstack_depth<=0:
                print('Already at highest layer of call stack')
                self.callstack_depth = 0
            else:
                self.callstack_depth = self.callstack_depth -1
        else:
            self.callstack_depth = len(self.callstack) - 1

        call = self.callstack[self.callstack_depth]

        call_function = call.fun_name
        execution_index = self.get_execution_index(call)
        arguments = ''

        surrounding_lines = list()

        while execution_index>0:
            if self.execution_log[execution_index].event=='line' and self.execution_log[execution_index].fun_name == call_function:
                surrounding_lines.append((self.execution_log[execution_index].rel_lineno,self.execution_log[execution_index].code))
            if self.execution_log[execution_index].event=='line' and self.execution_log[execution_index].rel_lineno == 2 and self.execution_log[execution_index].fun_name == call_function:
                surrounding_lines.append((self.execution_log[execution_index-1].rel_lineno,' call: '+self.execution_log[execution_index-1].fun_name))
                break
            if self.execution_log[execution_index].event=='call' and self.execution_log[execution_index].fun_name == call_function:
                arguments = str(self.execution_log[execution_index].changed_vars)
                break
            execution_index = execution_index - 1

        surrounding_lines.reverse()
        index = 0
        print(str(call_function) + '('+str(arguments)+')')
        for lineno,code in surrounding_lines:
            if index == len(surrounding_lines)-1:
                print('>> '+str(lineno)+'\t'+str(code))
            else:
                print(str(lineno)+'\t'+str(code))
            index = index + 1


    def down_command(self,arg=""):
        """Move down the call stack towards the callees"""

        print('Current depth: '+str(self.callstack_depth))
        if(self.callstack):
            if self.callstack_depth<len(self.callstack)-1:
                self.callstack_depth = self.callstack_depth + 1
                call = self.callstack[self.callstack_depth]
                call_function = call.fun_name
                execution_index = self.get_execution_index(call)
                surrounding_lines = list()

                while execution_index>0:
                    if self.execution_log[execution_index].event=='line' and self.execution_log[execution_index].fun_name == call_function:
                        surrounding_lines.append((self.execution_log[execution_index].rel_lineno,self.execution_log[execution_index].code))
                    if self.execution_log[execution_index].event=='line' and self.execution_log[execution_index].rel_lineno == 2 and self.execution_log[execution_index].fun_name == call_function:
                        surrounding_lines.append((self.execution_log[execution_index-1].rel_lineno,' call: '+self.execution_log[execution_index-1].fun_name))
                        break
                    execution_index = execution_index - 1

                surrounding_lines.reverse()
                index = 0
                print(' ')
                for lineno,code in surrounding_lines:
                    if index == len(surrounding_lines)-1:
                        print('>> '+str(lineno)+'\t'+str(code))
                    else:
                        print(str(lineno)+'\t'+str(code))
                    index = index + 1
            else:
                print('Already on lowest layer of call stack, out of range')
        else:
            print('Already on lowest layer of call stack, no callstack yet')

    def backconstruct_vars(self):
        backconstructed_vars = dict()
        if self.current_step>0:
            num_subroutines = 0
            for i in range(0,self.current_step):
                current_execution = self.execution_log[self.current_step-(i+1)]
                if current_execution.event == 'return':
                    num_subroutines  +=1
                elif current_execution.event == 'call':
                    if num_subroutines == 0: break
                    num_subroutines -=1
                elif num_subroutines==0:
                    changed_vars = current_execution.changed_vars.copy()
                    changed_vars.update(backconstructed_vars)
                    backconstructed_vars = changed_vars

            self.variables = backconstructed_vars
            return  True
        else:
            self.variables=self.execution_log[0].changed_vars.copy()
            return False

    def construct_vars_at(self,target_step):
        constructed_vars = dict()

        if target_step>0:
            num_subroutines = 0
            for i in range(0,target_step+1):
                current_execution = self.execution_log[target_step-i]
                if current_execution.event == 'return':
                    if i>0:
                        num_subroutines  +=1
                elif current_execution.event == 'call':
                    if num_subroutines == 0:
                        changed_vars = current_execution.changed_vars.copy()
                        changed_vars.update(constructed_vars)
                        constructed_vars = changed_vars
                        break
                    num_subroutines -=1
                elif num_subroutines==0:
                    changed_vars = current_execution.changed_vars.copy()
                    changed_vars.update(constructed_vars)
                    constructed_vars = changed_vars
        else:
            self.variables=self.execution_log[0].changed_vars.copy()
        return constructed_vars

    def construct_vars(self):
        event = self.current_state.event
        if event == 'return':
            self.variables.clear() #TODO this deletes too early, probably fine now not sure
        elif event == 'call':
            self.variables = self.current_state.changed_vars.copy()
        else:
            self.variables.update(self.current_state.changed_vars)

    def interaction_loop(self):
        self.interact = True
        while self.interact:
            command = input("(debugger) ")
            self.execute(command)

    def commands(self):
        cmds = [method.replace('_command', '')
                for method in dir(self.__class__)
                if method.endswith('_command')]
        cmds.sort()
        return cmds

    def command_method(self, command):
        if command.startswith('#'):
            return None  # Comment

        possible_cmds = [possible_cmd for possible_cmd in self.commands()
                         if possible_cmd==command]
        if len(possible_cmds) != 1:
            self.help_command(command)
            return None

        cmd = possible_cmds[0]
        return getattr(self, cmd + '_command')

    def execute(self, command):
        sep = command.find(' ')
        if sep > 0:
            cmd = command[:sep].strip()
            arg = command[sep + 1:].strip()
        else:
            cmd = command.strip()
            arg = ""

        method = self.command_method(cmd)
        if method:
            method(arg)

    def next_command(self,arg=""):
        """Step over function calls going to the next line"""
        #TODO adjust loop with callstack depth or similar (recursion issues)
        if self.current_step<len(self.execution_log)-1:
            if self.execution_log[self.current_step+1].event=="call":
                self.step()
                fun_name = self.current_state.fun_name
                while not (self.current_state.event == 'return' and self.current_state.fun_name == fun_name):
                    self.step()
                    self.variables = self.construct_vars_at(self.current_step)
                    if self.has_breakpoint():
                        self.check_watchpoints()
                        self.print_debugger_status()
                        return

        self.step_to_line()
        self.check_watchpoints()
        self.print_debugger_status()

    def previous_command(self,arg=""):
        """Step over function calls going to the previous line"""
        #TODO adjust loop with callstack depth or similar (recursion issues)
        if self.current_step>1:
            fun_name = self.current_state.fun_name
            if self.execution_log[self.current_step-1].event=="return":
                self.backstep_to_line()
                while self.current_state.fun_name != fun_name:
                    if self.has_breakpoint(True):
                        break
                    self.backstep_to_line()
            else:
                self.backstep_to_line()

        self.check_watchpoints()
        self.print_debugger_status()

    def next_state(self):
        if self.current_step<len(self.execution_log)-1:
            return self.execution_log[self.current_step+1]
        else:
            return None

    def lines_above(self,num_lines):
        surrounding_lines = list()
        fun_code = self.current_state.fun_code
        if fun_code:
            index = self.current_state.rel_lineno -1

            for i in range(max(0,index-num_lines),index):
                surrounding_lines.append(fun_code[i])
        return surrounding_lines


    def lines_below(self,num_lines):
        surrounding_lines = list()
        fun_code = self.current_state.fun_code
        self.log(fun_code)
        if fun_code:
            index = self.current_state.rel_lineno - 1
            if index<len(fun_code)-1:
                for i in range(index+1, min(len(fun_code),index+num_lines+1)):
                    surrounding_lines.append(fun_code[i])
        return surrounding_lines



    def test_command(self,arg=""):
        #self.lines_below(int(arg))
        surrounding_lines = self.lines_below(int(arg))
        surrounding_lines = surrounding_lines + [str(self.current_state.abs_lineno)+(">> "+ str(self.current_state.code))]
        for surrounding_line in surrounding_lines:
            print(str(surrounding_line))

    def list_command(self,args=""):
        """Print the source code around the current line (with the current line marked)"""
        arg_iterator = map(int, re.findall(r'\d+', args))

        if args!="" and re.match('^[0-9 |' ']*$',args):

             parsed_args = list()

             for parsed_arg in arg_iterator:
                 parsed_args.append(parsed_arg)
             surrounding_lines = list()

             if len(parsed_args) == 0:

                 surrounding_lines = self.lines_above(2)
                 surrounding_lines = surrounding_lines + [str(self.current_state.abs_lineno)+(">> "+ str(self.current_state.code))]
                 surrounding_lines = surrounding_lines + self.lines_below(2)
                 for surrounding_line in surrounding_lines:
                      print(str(surrounding_line))

             elif len(parsed_args) == 1:
                 surrounding_lines = surrounding_lines + (self.lines_above(parsed_args[0]))
                 surrounding_lines = surrounding_lines + [str(self.current_state.abs_lineno)+(">> " + str(self.current_state.code))]
                 surrounding_lines = surrounding_lines + (self.lines_below(parsed_args[0]))
                 for surrounding_line in surrounding_lines:
                      print(str(surrounding_line))

             elif len(parsed_args) == 2:
                 surrounding_lines  = surrounding_lines + (self.lines_above(parsed_args[0]))
                 surrounding_lines = surrounding_lines + [str(self.current_state.abs_lineno)+(">> " + str(self.current_state.code))]
                 surrounding_lines  = surrounding_lines + (self.lines_below(parsed_args[1]))
                 for surrounding_line in surrounding_lines:
                      print(str(surrounding_line))
             else:
                self.log("Wrong usage: list <above> <below> :Print <above> lines before and <below> lines after the current line")

        elif args.replace(" ","")== "":
             surrounding_lines = self.lines_above(2)
             surrounding_lines = surrounding_lines + [str(self.current_state.abs_lineno)+(">> "+ str(self.current_state.code))]
             surrounding_lines = surrounding_lines + self.lines_below(2)
             for surrounding_line in surrounding_lines:
                  print(str(surrounding_line))
        else:
            self.log("Wrong usage: list <above> <below> :Print <above> lines before and <below> lines after the current line")


    def until_command(self,arg=""):
        """Execute forwards until a certain point.
        'until' executes until a line greater than the current line is reached
        'until <line_number>' executes until a line greater than <line_number> is reached
        'until <filename>:<line_number>' executes until line <line_number> is reached in file <filename>
        'until <function_name>' executes until function <function_name> is called
        'until <filename>:<function_name>' executes until function <function_name> declared in <filename> is called
        """
        #TODO special case no arg act as next
        if not arg:
            #no arg case
            self.until_line(self.current_state.abs_lineno,self.current_state.file)
            return

        function_name=""
        target_line=0
        sep = arg.find(':')
        if sep > 0:
            #compound arg case
            filename = arg[:sep].strip()
            secondary_arg = arg[sep+1:].strip()
            if secondary_arg.isdigit():
                target_line=int(secondary_arg)
            else: 
                function_name = secondary_arg

            if target_line > 0:
                self.until_line(target_line,filename)
            else:
                self.until_function(function_name,filename)
        else:
            #simple arg case
            if arg.isdigit():
                target_line=int(arg)
            else:
                function_name=arg

            if function_name:
                self.until_function(function_name,self.current_state.file)
            else:
                self.until_line(target_line,self.current_state.file)


    def backuntil_command(self,arg=""):
        """Execute backwards until a certain point.
        'backuntil' executes backwards until a line less than the current line is reached
        'backuntil <line_number>' executes backwards until a line less than <line_number> is reached
        'backuntil <filename>:<line_number>' executes backwards until line <line_number> is reached in file <filename>
        'backuntil <function_name>' executes backwards until function <function_name> is called
        'backuntil <filename>:<function_name>' executes backwards until function <function_name> declared in <filename> is called
        """
        if not arg:
            #no arg case
            self.backuntil_line(self.current_state.abs_lineno,self.current_state.file)
            return

        function_name=""
        target_line=0
        sep = arg.find(':')
        if sep > 0:
            #compound arg case
            filename = arg[:sep].strip()
            secondary_arg = arg[sep+1:].strip()
            if secondary_arg.isdigit():
                target_line=int(secondary_arg)
            else:
                function_name = secondary_arg
            if target_line > 0:
                self.backuntil_line(target_line,filename)
            else:
                self.backuntil_function(function_name,filename)
        else:
            #simple arg case
            if arg.isdigit():
                target_line=int(arg)
            else:
                function_name=arg

            if function_name:
                self.backuntil_function(function_name,self.current_state.file)
            else:
                self.backuntil_line(target_line,self.current_state.file)

    def until_function(self,target_function,target_file):
        """Advance until call to target_function and in target_file if given"""
        stepped = True
        while not (self.current_state.fun_name == target_function and self.current_state.event=="call"
                   and self.current_state.file==target_file ):
            stepped = self.step()
            self.variables=self.construct_vars_at(self.current_step)
            if not stepped:
                break
            if self.has_breakpoint():
                self.check_watchpoints()
                self.print_debugger_status()
                return

        if stepped:
            self.step_to_line()
        else:
            self.backstep_to_line()
        self.check_watchpoints()
        self.print_debugger_status()


    def backuntil_function(self,target_function,target_file):
        """Run backwards until call to target_function and in target_file if given"""
        while not (self.current_state.fun_name == target_function and self.current_state.event=="call" and
                    (self.current_state.file==target_file)):
            stepped = self.step_back()
            if not stepped:
                break

            self.variables=self.construct_vars_at(self.current_step)
            if self.has_breakpoint(True):
                self.check_watchpoints()
                self.print_debugger_status()
                return

        self.step_to_line()
        self.check_watchpoints()
        self.print_debugger_status()


    def until_line(self,target_line,target_file):
        """Advance until program line is greater than target_line and in target_file if given"""
        if target_line<=0:
            self.log("Invalid line: " + str(target_line))
            return

        for i in range(self.current_step,len(self.execution_log)):
            if self.current_state.abs_lineno>target_line and self.current_state.file==target_file:
                break
            self.step_to_line()
            if self.has_breakpoint():
                break

        self.check_watchpoints()
        self.print_debugger_status()


    def backuntil_line(self,target_line,target_file):
        """Run backwards until program line is lower than target_line and in target_file"""
        if target_line<=1:
            self.log("Invalid line: " + str(target_line))
            return

        for i in range(0,self.current_step+1):
            if self.current_state.abs_lineno<target_line and self.current_state.file==target_file:
                break
            self.backstep_to_line()
            if self.has_breakpoint(True):
                break

        self.check_watchpoints()
        self.print_debugger_status()


    def finish_command(self,arg=""):
        """Execute until return of current function"""
        #TODO adjust loop with callstack depth or similar (recursion issues)
        fun_name = self.current_state.fun_name
        while not (self.current_state.fun_name == fun_name and self.current_state.event=="return"):
            self.step()
            self.variables = self.construct_vars_at(self.current_step)
            if self.has_breakpoint():
                self.check_watchpoints()
                self.print_debugger_status()
                return

        self.backstep_to_line()
        self.check_watchpoints()
        self.print_debugger_status()

    def start_command(self,arg=""):
        """Execute backwards until a function start"""
        #TODO adjust loop with callstack depth or similar (recursion issues)
        fun_name = self.current_state.fun_name
        while not (self.current_state.fun_name == fun_name and self.current_state.event=="call"):
            self.step_back()
            self.variables = self.construct_vars_at(self.current_step)
            if self.has_breakpoint(True):
                self.check_watchpoints()
                self.print_debugger_status()
                return
        self.step_to_line()
        self.check_watchpoints()
        self.print_debugger_status()

    def has_breakpoint(self,backwards=False):
        active_breakpoints = [bp for bp in self.breakpoints if self.breakpoints[bp].break_here(self.current_step,self.execution_log,self.variables,backwards)]
        return len(active_breakpoints)>0

    def quit_command(self,arg=""):
        """End the debugger session"""
        self.interact=False
        self.log("Quit debugger session")

    def continue_command(self, arg=""):
        """ Continue execution forward until a breakpoint is hit, or the program finishes"""
        stepping = True
        while stepping:
            stepped = self.step_to_line()
            if (not stepped) or self.has_breakpoint():
                stepping = False

        self.check_watchpoints()
        self.print_debugger_status()

    def reverse_command(self, arg=""):
        """Continue execution backward until a breakpoint is hit, or the program starts"""
        stepping = True
        while stepping:
            stepped = self.backstep_to_line()
            if not stepped or self.has_breakpoint(True):
                stepping = False
        self.check_watchpoints()
        self.print_debugger_status()

    def backstep_command(self,arg=""):
        """Step to the previous executed line"""
        self.backstep_to_line()
        self.check_watchpoints()
        self.print_debugger_status()

    def step_command(self, arg=""):
        """Execute up to the next line"""
        self.step_to_line()
        self.check_watchpoints()
        self.print_debugger_status()

    def print_command(self, arg=""):
        """Print an expression. If no expression is given, print all variables"""
        #vars = self.current_state.frame.f_locals
        vars = self.variables
        if not arg:
            self.log("\n".join([f"{var} = {repr(vars[var])}" for var in vars]))
        else:
            try:
                self.log(f"{arg} = {repr(eval(arg, None, vars))}")
            except Exception as err:
                self.log(f"{err.__class__.__name__}: {err}")

    def break_command(self, arg=""):
        """Set a breakpoint in given line. If no line is given, list all breakpoints"""
        if arg:
            if len(self.breakpoints)>0:
                largest_key= max(self.breakpoints.keys())
            else:
                largest_key = 0

            if arg.isdigit():
                self.breakpoints[largest_key+1]=Breakpoint("line",int(arg),None,None,self.current_state.file)
            else:
                sep = arg.find(':')
                if sep>0:
                    file_name = arg[:sep].strip()
                    func_name = arg[sep+1:].strip()
                    self.breakpoints[largest_key+1]=Breakpoint("func",None,func_name,None,file_name)
                else:
                    self.breakpoints[largest_key+1]=Breakpoint("func",None,arg,None,self.current_state.file)
        else:
            self.log("Missing argument, use as 'break <line>|<function_name>|<file_name>:<function_name>'")

    def delete_command(self, arg=""):
        """Delete breakpoint with given <breakpoint_id>"""
        if arg:
            breakpoint_id = self.breakpoint_lookup(arg)
            if breakpoint_id==-1:
                self.log("No such breakpoint: "+arg)
            else:
                self.breakpoints.pop(breakpoint_id)
        else:
            self.log("Missing argument, use as 'delete <breakpoint_id>'")

    def breakpoints_command(self,arg=""):
        """Display all available breakpoints"""
        for breakpoint in self.breakpoints:
            alias = self.breakpoints[breakpoint].alias
            if alias:
                self.log(alias+ "   " +str(self.breakpoints[breakpoint]))
            else:
                self.log(str(breakpoint)+ "   " +str(self.breakpoints[breakpoint]))

    def disable_command(self,arg=""):
        """Use as 'disable <breakpoint_id>'    Suspend  breakpoint with the index <breakpoint_id>"""
        if arg:
            breakpoint_id = self.breakpoint_lookup(arg)
            if breakpoint_id==-1:
                self.log("No such breakpoint: "+arg)
            else:
                self.breakpoints[breakpoint_id].disable()
        else:
            self.log("Missing argument, use as 'disable <breakpoint_id>'")

    def enable_command(self,arg=""):
        """Use as 'enable <breakpoint_id>'    Re-enable a breakpoint with the index <breakpoint_id>"""
        if arg:
            breakpoint_id = self.breakpoint_lookup(arg)
            if breakpoint_id==-1:
                self.log("No such breakpoint: "+arg)
            else:
                self.breakpoints[breakpoint_id].enable()
        else:
            self.log("Missing argument, use as 'enable <breakpoint_id>'")

    def breakpoint_lookup(self,arg):
        if arg.isdigit():
            if int(arg) in self.breakpoints.keys():
                return int(arg)

        else:
            for breakpoint_id in self.breakpoints:
                if self.breakpoints[breakpoint_id].alias==arg:
                    return breakpoint_id
        return -1

    def alias_command(self,arg=""):
        """Use as 'alias <breakpoint_id> <breakpoint_name>'  Set the alias <breakpoint_name> for breakpoint <breakpoint_id> """
        if not arg:
            self.log("Requires arguments <breakpoint_id> <breakpoint_name>")

        sep = arg.find(' ')
        if sep > 0:
            breakpoint_id = arg[:sep].strip()
            alias = arg[sep+1:].strip()
            if not alias:
                self.log("Empty string not allowed as alias")
                return
            if breakpoint_id.isdigit():
                breakpoint_aliases=[bp.alias for bp in self.breakpoints.values()]
                if alias in breakpoint_aliases:
                    self.log("Alias already exists: "+ alias)
                    return
                try:
                    breakpoint = self.breakpoints[int(breakpoint_id)]
                    breakpoint.alias=alias
                except KeyError:
                    self.log("No breakpoint with id: " + breakpoint_id)
            else:
                self.log("Invalid arguments. Use as 'alias <breakpoint_id> <breakpoint_name>'")

        else:
            self.log("Invalid arguments. Use as 'alias <breakpoint_id> <breakpoint_name>'")


    def cond_command(self,arg=""):
        """Use as 'cond <line> <condition>'    Set a breakpoint at which the execution is stopped at line <line>
        if a condition <condition> is true"""
        if arg:
            sep = arg.find(' ')
            if sep>0:
                line = arg[:sep].strip()
                cond_expression = arg[sep+1:].strip()
                if not line.isdigit():
                    self.log("Faulty argument, use as 'cond <line> <condition>'")
                    return

                if len(self.breakpoints)>0:
                    largest_key= max(self.breakpoints.keys())
                else:
                    largest_key = 0

                self.breakpoints[largest_key+1]=Breakpoint("cond",int(line),None,cond_expression,self.current_state.file)
            else:
                self.log("Faulty argument, use as 'cond <line> <condition>'")
        else:
            self.log("Missing argument, use as 'cond <line> <condition>'")

    def watch_command(self,arg=""):
        """Creates a numbered watchpoint for the given variable"""
        if arg:
            variables = [var for (var, val) in self.watchpoints.values()]
            if not arg in variables:
                if len(self.watchpoints)>0:
                    largest_key= max(self.watchpoints.keys())
                else:
                    largest_key = 0
                self.watchpoints[largest_key+1]=(arg,None)

            else:
                self.log("Watchpoint already exists: "+arg)
        else:
            self.print_watchpoints()

    def unwatch_command(self,arg=""):
        """Remove a watchpoint"""
        if arg:
            if arg.isdigit():
                try:
                    self.watchpoints.pop(int(arg))
                except KeyError:
                    self.log("No such watchpoint: "+ arg)
                self.log("Watchpoints", self.watchpoints)
            else:
                self.log("The argument has to be an integer: unwatch <watch_id>")
        else:
            self.log("Specify which watchpoint to remove: unwatch <watch_id>")
            self.log("Watchpoints", self.watchpoints)

    def update_watchpoints(self,print_changes=False):
        changes={}
        for watch_id in self.watchpoints:
            entry = self.watchpoints[watch_id]
            if entry[0] in self.variables:
                if self.variables[entry[0]]!=entry[1]:
                    changes[watch_id]=(entry[0],self.variables[entry[0]])
        self.watchpoints.update(changes)
        if print_changes and changes:
            #self.log("Changed watchpoints: " + repr(changes))
            changes_s= ", ".join([repr(self.watchpoints[watch_id][0])+" = "+ repr(self.watchpoints[watch_id][1])
                               for watch_id in changes])
            self.log("Changed watchpoints: " + changes_s)
        return changes
        #repr(eval(self.current_state.code, None, self.variables))

    def check_watchpoints(self):
        if not self.watchpoints:
            return

        if self.current_step>0:
            if self.execution_log[self.current_step-1].event == 'return':
                var_backup = self.variables.copy()
                step = self.current_step-1
                state =self.execution_log[step]
                while not (step == 0 or (state.event=='line' and state.fun_name == self.current_state.fun_name)):
                    step -= 1
                    state = self.execution_log[step]
                if step != 0:
                    self.log(str(state))
                    self.variables = self.construct_vars_at(step)
                    self.update_watchpoints()
                    self.variables=var_backup
                    self.update_watchpoints(True)
            else:
                self.update_watchpoints(True)
        else:
            self.update_watchpoints(True)

    def print_watchpoints(self):
        watched_vars= ", ".join([str(watch_id) + ": "+repr(self.watchpoints[watch_id][0])
                               for watch_id in self.watchpoints])
        self.log(watched_vars)


    def print_debugger_status(self):
        frame = self.current_state.frame
        event = self.current_state.event
        changes = self.current_state.changed_vars
        changes_s = ", ".join([var + " = " + repr(changes[var])
                               for var in changes])

        if event == 'call':
            self.log("This is a call event, which should never be printed")
            #self.log(repr(self.current_state.abs_lineno) + "    Calling " + self.current_state.fun_name + '(' + changes_s + ')')

        if event == 'line':
            current_line = self.current_state.code
            self.log(repr(self.current_state.abs_lineno) + '    ' + current_line)
        if event == 'return':
            self.log("This is a return event, which should never be printed")
            #self.log(repr(self.current_state.abs_lineno) +'    '+ self.current_state.fun_name + '()' + " returns " + repr(self.current_state.code)) #used to have +repr(arg)


    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        print(*objects, sep=sep, end=end, file=sys.stdout, flush=True)

    def help_command(self, command=""):
        """Give help on given command. If no command is given, give help on all"""
        if command:
            possible_cmds = [possible_cmd for possible_cmd in self.commands()
                             if possible_cmd.startswith(command)]

            if len(possible_cmds) == 0:
                print(f"Unknown command {repr(command)}. Possible commands are:")
                possible_cmds = self.commands()
            elif len(possible_cmds) > 1:
                print(f"Ambiguous command {repr(command)}. Possible expansions are:")
        else:
            possible_cmds = self.commands()

        for cmd in possible_cmds:
            method = self.command_method(cmd)
            print(f"{cmd:10} -- {method.__doc__}")

In [ ]:
def foo():
    a = 12
    a= 13
    a = bar(21)
    x = 5
    recursion(3)
    #b = fun(a,x)
    a = 11

    return

def bar(arg):
    b = arg
    a = 7
    return b
    #code()

def code():
    return

def recursion(value):
    if value>0:
        recursion(value - 1)

In [ ]:
with Debugger():
     pass



In [ ]:
#d= Debugger(res)
#d.start()
#next_inputs(["step","print","step","step","print","step","print s","continue"])
#next_inputs(["step","step","step","step","step","step","continue"])
#next_inputs(["step","break 4","continue","continue"])
#with Debugger():
#    remove_html_markup("<b>bold</b>")

In [ ]:
def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style, layout=widgets.Layout(height='auto', width='auto'))


play = widgets.Play(
#     interval=10,
    value=0,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
tester = widgets.HBox([play, slider])


top_left_button = create_expanded_button("Top left", 'info')
top_right_button = create_expanded_button("Top right", 'success')
bottom_left_button = create_expanded_button("Bottom left", 'danger')
bottom_right_button = create_expanded_button("Bottom right", 'warning')

top_left_text = widgets.IntText(description='Top left', layout=widgets.Layout(width='auto', height='auto'))
top_right_text = widgets.IntText(description='Top right', layout=widgets.Layout(width='auto', height='auto'))
bottom_left_slider = widgets.IntSlider(description='Bottom left', layout=widgets.Layout(width='auto', height='auto'))
bottom_right_slider = widgets.IntSlider(description='Bottom right', layout=widgets.Layout(width='auto', height='auto'))

header_button = create_expanded_button('Header', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')

widgets.AppLayout(header=tester,
          center=center_button,
          right_sidebar=right_button,
          pane_widths=[2, 0.5, 1],
          pane_heights=[1, 17, 1])


